<a href="https://colab.research.google.com/github/Wattsy2020/DeepLearningA1/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP5329 Assignment 1

Team Member
- Mirope Yuhao Hu, SID: 470139936
- Sean Hongbo Du,	SID: 500635346
- Liam Watts, SID: 510562348


### Liam Watts: 1, 2, 6 - More than 1 hidden layer, ReLU Activation, Softmax output layer and cross entroppy loss
### Mirope: 4, 3, 8 - Momentum in SGD, weight decay, batch normalization
### Sean: 5, 7, 0 - dropout, mini-batch training, data preprocess

## How to run the code

## All necessary Imports

__NOT__ allowed:
1. use Deep Learning frameworks (e.g. PyTorch, Tensorflow, Caffe, and KERAS)
2. any kinds of auto-grad tools (e.g. autograd)
3. sklearn

In [ ]:
import numpy as np

## Data Loading/Preprocessing

In [ ]:
def oneHot_encode():
    pass

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load data -> subject to change for the final version
dir_name = '/content/drive/MyDrive/Colab Notebooks/COMP5329/Ass1/Assignment1-Dataset/'

test_data = np.load(dir_name + 'test_data.npy')
test_label = np.load(dir_name + 'test_label.npy')
train_data = np.load(dir_name + 'train_data.npy')
train_label = np.load(dir_name + 'train_label.npy')

print('Shape of original train data is:\t', train_data.shape,
      '\nShape of original train label is:\t', train_label.shape,
      '\nShape of original test data is:\t\t', test_data.shape,
      '\nShape of original test label is:\t', test_label.shape )

# print('\nAll labels are:\t\t', list(np.unique(test_label)))

# # preprocessing: normalization might be unnecessary
# # test_x = Normalizer().fit_transform(test_data)
# # train_x = Normalizer().fit_transform(train_data)
# test_x = test_data
# train_x = train_data
# test_y = OneHotEncoder().fit_transform(test_label).toarray()
# train_y = OneHotEncoder().fit_transform(train_label).toarray()

# print('\nShape of transformed train label is:\t', train_y.shape,
#       '\nShape of transformed train label is:\t', test_y.shape )

Shape of original train data is:	 (50000, 128) 
Shape of original train label is:	 (50000, 1) 
Shape of original test data is:		 (10000, 128) 
Shape of original test label is:	 (10000, 1)


## Algorithms / Structures

__Requirements__
1. More than one hidden layer
2. ReLU activation
3. Weight decay
4. Momentum in SGD
5. Dropout
6. Softmax and cross-entropy 
7. Mini-batch training
8. Batch Normalization

In this part, try to implement each layer/optimiser/function as modules

In [ ]:
# Liam Watts: 1, 2, 6 - activation, layer, softmax
# Mirope: 4, 3, 8 - Momentum in SGD, weight decay, batch normalization
# Sean: 5, 7, 0 - dropout, mini-batch training, data preprocess

In [ ]:
dlogsigmoid(sigmoid(np.array([[1, 2, 3], [4, 5, 6]])))

array([[0.26894142, 0.11920292, 0.04742587],
       [0.01798621, 0.00669285, 0.00247262]])

In [ ]:
# Define functions required for the Multi Layer Perceptron
def he_uniform(input_size=1, output_size=1):
    '''outputs a matrix of size (output_size, input_size) with random initialization'''
    r = np.sqrt(6/input_size)
    return np.random.uniform(-r, r, size=(output_size, input_size))

def relu(matrix):
    '''return the elementwise ReLU activation of the matrix'''
    return np.maximum(matrix, 0)

def drelu(matrix):
    '''
    parameters:
        matrix: the relu activations
    
    outputs:
        the derivative for relu on every element of the matrix
    '''
    return np.minimum(np.maximum(matrix, 0), 1) # this defines the derivative of relu at 0 as 0 to tf.nn.relu()'s standard (which ensures a sparser gradient matrix)

# define the sigmoid function for testing, it's easier to implement backprop on sigmoid than softmax
def sigmoid(matrix):
    '''return sigmoid on every element of the matrix'''
    return 1/(1 + np.power(np.e, -matrix))

def dlogsigmoid(matrix):
    '''
    parameters:
        matrix: the sigmoid activations
    
    outputs:
        the derivative for log sigmoid on every element of the matrix
    '''
    return (1 - matrix) # dsigmoid = sigmoid(1 - sigmoid)  so dlog(sigmoid) = sigmoid(1 - sigmoid)/sigmoid = (1 - sigmoid)

def softmax(matrix):
    '''return the softmax activation of a matrix (each row vector has softmax applied to it)'''
    matrix = np.power(np.e, matrix)
    total = np.sum(matrix, axis=1).reshape(matrix.shape[0], 1)
    return matrix/total # here we broadcast total (a vector) to divide each row in the matrix, broadcasting is frequently used to vectorize our code

# it is faster to calculate the derivative of log(softmax(vector)), and because we use cross entropy as the loss function
# the only derivative we need to calculate is this log(softmax(vector))
def dlogsoftmax(vector):
    '''
    parameters:
        vector: the softmax activations
    
    outputs:
        the jacobian, a matrix of derivatives of log(vector[i]) wrt vector[j]
    '''
    pass # to implement (examine this https://stackoverflow.com/questions/35304393/trying-to-understand-code-that-computes-the-gradient-wrt-to-the-input-for-logsof)

In [ ]:
class MLP:
    ''' 
    Implements a Multi Layer Perceptron, including both the feed-forward phase and backward propagation
    The output activation must be softmax, and the loss function cross entropy

    MLP.__init__()
        parameters:
            hidden: an array with the number of neurons for each layer
        
        initializes weight matrices with shape (output_size, input_size) to match Pytorch's standard
    '''
    def __init__(self, hidden=[1], input_size=1, output_size=1, initializer=he_uniform):
        '''initialize the weight and bias matrices for each layer'''
        self.weights = []
        self.biases = []
        for input_size_i, output_size_i in zip([input_size] + hidden, hidden + [output_size]):
            self.weights.append(initializer(input_size=input_size_i, output_size=output_size_i))
            self.biases.append(np.zeros((output_size_i, 1)))

    def forward(self, X):
        '''
        calculate output of the network, storing the hidden states in self.activations (activations[0] is the feature input)
        
        parameters:
            X: the input matrix of shape (batch_size, input_size)

        returns:
            yhat: the prediction matrix of shape (batch_size, output_size)
        '''
        self.activations = [X]
        for w, b in zip(self.weights, self.biases):
            activation = self.forward_one_layer(self.activations[-1], w, b)
            self.activations.append(activation)
        return softmax(self.activations[-1])

    def forward_one_layer(self, inputs, weights, biases):
        '''
        compute the output of a layer for certain inputs, return the output
        
        we follow the pytorch standard, so shapes are as follows:
        inputs: a matrix of shape (batch_size, feature_size)
        weights: a matrix of shape (output_size, feature_size)
        biases: a matrix of shape (output_size, 1)
        returns: a matrix of shape (batch_size, output_size)
        '''
        return relu(np.dot(inputs, weights.T) + biases.T)

    def backprop(self, y):
        '''given the true outputs y perform one step of gradient descent'''
        assert self.hidden != [] # we need the hidden states
        # get the derivative of cross-entropy loss for the output

        # do backprop layer by layer, passing to each layer the derivative of loss w.r.t that layer's outputs
            # while simultaneously updating the weights

In [ ]:
mlp = MLP(hidden=[64], input_size=128, output_size=10)
preds = mlp.forward(train_data)
preds, np.sum(preds, axis=1)

(array([[0.07955625, 0.05938695, 0.12384888, ..., 0.13039277, 0.05938695,
         0.05938695],
        [0.08884543, 0.11836953, 0.04426743, ..., 0.1514843 , 0.07748669,
         0.04426743],
        [0.02641715, 0.66916231, 0.03463996, ..., 0.02641715, 0.04047708,
         0.05971798],
        ...,
        [0.01672059, 0.8305105 , 0.01672059, ..., 0.01672059, 0.01672059,
         0.01707037],
        [0.0114723 , 0.60479851, 0.0116819 , ..., 0.00958395, 0.20692705,
         0.03454694],
        [0.15077616, 0.08481901, 0.0352657 , ..., 0.03517824, 0.39013562,
         0.15460947]]), array([1., 1., 1., ..., 1., 1., 1.]))

## Experiments

In [ ]:
# the baseline model
def baseline_model():
	# create model
    model = Sequential()
    model.add(Dense(512, input_dim=128, activation='relu')) # 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1)) # 5
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1)) # 5
    model.add(BatchNormalization()) # 8
    model.add(Dense(10, activation='softmax')) # 1, 6.1
    opt = SGD(lr=0.3, momentum=0.9, decay=0.001) # optimizers: 3, 4
    # 6.2
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

  
# Crossing Validation
def training(model, epoches=20, batch_size=256):
  for i in epoches:
    